<a href="https://colab.research.google.com/github/Towshin05/Codesoft_Internship/blob/master/SVM_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!kaggle datasets download -d shantanudhakadd/bank-customer-churn-prediction

Dataset URL: https://www.kaggle.com/datasets/shantanudhakadd/bank-customer-churn-prediction
License(s): other
  0% 0.00/262k [00:00<?, ?B/s]
100% 262k/262k [00:00<00:00, 86.5MB/s]


In [2]:
import zipfile
zip_ref=zipfile.ZipFile('/content/bank-customer-churn-prediction.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [4]:
train_df=pd.read_csv('/content/Churn_Modelling.csv')
train_df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
train_df = pd.get_dummies(train_df, columns=['Geography', 'Gender'], drop_first=True)

In [6]:
train_df['Exited'].value_counts()

,count
Exited,
0,7963
1,2037


In [7]:
X=train_df.drop(['RowNumber','CustomerId','Surname','Exited'],axis=1)
y=train_df['Exited']

In [8]:
y.value_counts()

,count
Exited,
0,7963
1,2037


In [9]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [12]:
y_train.value_counts()

,count
Exited,
0,5547
1,1453


In [13]:
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=1)

# Fit and transform the training data
X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)

In [14]:
y_test.value_counts()

,count
Exited,
0,2416
1,584


In [15]:
y_resampled.value_counts()

,count
Exited,
1,5547
0,5547


In [16]:
X_resampled=X_resampled.to_numpy()
X_test=X_test.to_numpy()

In [17]:
y_resampled=y_resampled.to_numpy()
y_test=y_test.to_numpy()

In [18]:
from sklearn.preprocessing import StandardScaler

In [19]:
scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled=scaler.transform(X_resampled)
X_test_scaled=scaler.transform(X_test)

In [20]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV


In [ ]:
clf=svm.SVC(kernel='linear',degree=2)
clf.fit(X_train_scaled,y_resampled)

SVC(degree=2, kernel='linear')

In [ ]:
y_pred=clf.predict(X_test_scaled)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.71


In [ ]:
classification_rep = classification_report(y_test, y_pred)
print('Classification Report:' ,classification_rep)


Classification Report:               precision    recall  f1-score   support

           0       0.91      0.72      0.80      2416
           1       0.37      0.70      0.49       584

    accuracy                           0.71      3000
   macro avg       0.64      0.71      0.64      3000
weighted avg       0.80      0.71      0.74      3000



GridsearchCv

In [21]:
param_grid = {
    'C': [0.1, 1, 10],  # Regularization parameter
    'kernel': ['linear', 'rbf', 'poly'],  # Kernel type
    'degree': [2, 3, 4],  # Degree for polynomial kernel
    'gamma': ['scale', 'auto']  # Kernel coefficient

}

In [22]:

svm_model = svm.SVC()
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

In [23]:
grid_search.fit(X_train_scaled, y_resampled)


Fitting 5 folds for each of 54 candidates, totalling 270 fits


GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'degree': [2, 3, 4],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'poly']},
             verbose=2)

In [25]:
X_train_scaled.shape

(11094, 11)

In [24]:
y_resampled

array([1, 0, 0, ..., 1, 1, 1])

In [26]:
best_model = grid_search.best_estimator_

In [27]:
best_model.fit(X_train_scaled, y_resampled)

SVC(C=10, degree=2, gamma='auto')

In [28]:
val_pred=best_model.predict(X_test_scaled)

In [29]:
# Evaluate the model
accuracy = accuracy_score(y_test, val_pred)
print(f'Accuracy: {accuracy:.2f}')

# Optional: print a detailed classification report
print(classification_report(y_test, val_pred))

Accuracy: 0.78
              precision    recall  f1-score   support

           0       0.91      0.81      0.86      2416
           1       0.46      0.66      0.54       584

    accuracy                           0.78      3000
   macro avg       0.68      0.74      0.70      3000
weighted avg       0.82      0.78      0.80      3000



In [30]:

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, val_pred)

array([[1962,  454],
       [ 196,  388]])